In [1]:
from pyLM import *


import numpy as np
import os
import time as phys_time
from Bio import SeqIO


In [2]:
dir_path = '../output7/'

# Check if the directory exists
if not os.path.exists(dir_path):
    # The directory does not exist, create it
    os.makedirs(dir_path)
    print(f"Directory '{dir_path}' was created.")
else:
    # The directory exists
    print(f"Directory '{dir_path}' already exists.")


Directory '../output7/' was created.


In [3]:
# User defined Python scripts
import initiation as IC
import rxns_CME
import communicate
import species_counts
import hookSolver_CMEODE
import filesaving
import hook_CMEODE

In [4]:
#########################################################################################
def initializeCME(restartNum, restartInterval):
    """
    
    Decription: Set up the simulation time and intervals for CME simulation
    """
    # Initialize every restart CME simulation object

    sim=CME.CMESimulation(name="CMEODE_" + str(restartNum+1))
    # sim.setTimestep(ms(100))
    sim.setWriteInterval(1)
    sim.setHookInterval(1)
    # Every newly simulation is 60 seconds
    sim.setSimulationTime(restartInterval)
    
    return sim

#########################################################################################

In [5]:
%%capture captured_output

from datetime import datetime
start_time = datetime.now()

print('The simulation starts at '+ str(start_time))

print('The ID of this Process is ' + str(os.getpid()))

genomeFile3A =  '../input_data/syn3A.gb'
genome3A = next(SeqIO.parse(genomeFile3A, "gb"))

# Import genome
# Genome map will maintain in the whole cell cycle
# Initialize sim_properties, a dictionary record the genome information, trajectories, and other constants
sim_properties = {}

countsCSV = 'particle_counts.csv'
countsCSV_path = os.path.join(dir_path,countsCSV)

SACSV = 'SA.csv'
SACSV_path = os.path.join(dir_path, SACSV)

sim_properties['path'] = {'counts': countsCSV_path, 'SA': SACSV_path}

IC.initializeConstants(sim_properties, genome3A)


sim_properties['time_second'] = [int(0)]

LocusNumtoIndex = sim_properties['LocusNumtoIndex']

sim_properties['restartNum'] = []

restartInterval = 60 # s
sim_properties['restartInterval'] = restartInterval

endtime_minute = 105  # minutes

endtime_second = 6300 # seconds

restartNums = 105


for restartNum in range(0,restartNums):

    sim_properties['restartNum'].append(restartNum)
    print('*******************************************************************')
    print('Starting the simulation between ' + str(restartNum*restartInterval) +' to ' + str((restartNum+1)*restartInterval) + ' seconds')

    sim = initializeCME(restartNum,restartInterval)

    if restartNum == 0:
        IC.initializeMetabolitesCounts(sim_properties)
        IC.initializeMediumConcs(sim_properties)
        IC.initializeCosts(sim_properties)
        IC.initializeProteinMetabolitesCounts(sim_properties)
    

    # define the reactions and particles numbers every newly start simulation
    ini_start = phys_time.time()

    IC.addEnzymesCounts(sim, sim_properties)

    rxns_CME.addGeneticInformationReactions(sim, sim_properties, genome3A)

    IC.addGeneticInformationSpeciesCounts(sim,sim_properties)

    print('Initiation of CME reactions and counts in ' + str(phys_time.time() - ini_start) + ' second')

    break 

    # CMECounts need to be reinitialized per new CME simulation
    CMECounts = species_counts.SpeciesCounts(sim)
    hook = hookSolver_CMEODE.MyOwnSolver()

    hook.initializeSolver(sim, CMECounts, sim_properties, genome3A)



    if restartNum == 0:
        IC.initializeMembrane(sim_properties)
        # filesaving at simulation time = 0 second
        # filesaving.writeSAtoCSV(sim_properties['time_second'][-1], sim_properties)
        # filesaving.writeCountstoCSV(sim_properties['time_second'][-1], sim_properties)
    

    # Filename of the current simulation
    filename = os.path.join(dir_path, 'CME_ODE'+ str(restartNum+1) + '.lm')
    # filename = '../output/CMEODE_' + str(restartNum+1) + '.lm'
    # Remove lm file from last whole cell cycle simulation
    os.system("rm -rf %s"%(filename))
    

    # Filename of the last minute simulation
    #filename_last = '../output/CMEODE_' + str(restartNum) +'.lm'
    filename_last = os.path.join(dir_path, 'CME_ODE'+ str(restartNum) + '.lm')
    # Remove the lm file of last minute so that you will not get 105 files 
    # os.system("rm -rf %s"%(filename_last))
    
    save_start = phys_time.time()

    sim.save(filename)
    print('LM file Saved in ' + str(phys_time.time() - save_start) + ' second')

    runCMEODE_start = phys_time.time()
    
    # runCME_start = phys_time.time()
    
    sim.runSolver(filename = filename,solver = hook, replicates = 1)
    
    # Now the CME simulation ends, we use updateCMEcountsFile to read in the counts and pass them to ODE to do the last second's ODE simulation
    communicate.updateCMEcountsFile(sim, sim_properties, filename)
    
    hook_CMEODE.hook_CMEODE(sim_properties, genome3A)
    
    CSVfilesaving_start = phys_time.time()
    filesaving.writeSAtoCSV(restartNum, sim_properties)
    filesaving.writeCountstoCSV(restartNum,sim_properties)
    print('CSV File updated in ' + str(phys_time.time() - CSVfilesaving_start) + ' second')


    print('CMEODE run finished in ' + str(phys_time.time() - runCMEODE_start) + ' second')

    print('Finish the simulation between ' + str(restartNum*restartInterval) +' to ' + str((restartNum+1)*restartInterval) + ' second')
    print('*******************************************************************')

    if restartNum == 0:
        break



end_time = datetime.now()
print('The simulation ends at '+ str(end_time))



In [6]:
sim

Specie,Particle ID,Initial Count
RNAP,1,187
ribosomeP,2,500
Degradosome,3,120
High_Affinity_Site,4,16
High_Affinity_Bound,5,0
High_Affinity_Site_oriC,6,1
High_Affinity_Bound_oriC,7,0
Low_Affinity_Site_1,8,0
Low_Affinity_Site_2,9,0
Low_Affinity_Bound_1,10,0


In [ ]:
logfile = 'log.txt'
logfile_path = os.path.join(dir_path, logfile)

with open(logfile_path, "w") as f:
    f.write(captured_output.stdout)

In [ ]:
break

In [13]:
sim_properties['time_second']

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60]

In [ ]:
import pandas as pd
countsfile_path = '../output3/counts_tRNAcharging.csv'

if restartNum == 0:
    particleDF = pd.DataFrame()

    spec_IDs = []

    for specID in sim_properties['counts'].keys():
        spec_IDs.append(specID)
    
    particleDF['Time'] = spec_IDs

    for simulationTime in range(restartNum*60, (restartNum+1)*60+1):
        # 0, 1, 2, ..., 60 second
        new_counts = []
        for count in sim_properties['counts'].values():
            # Certain counts are not updated per second so use try-expect to avoid list index out of range
            try:
                new_counts.append(count[simulationTime])
            except:
                new_counts.append(count[-1])

        particleDF[simulationTime] = new_counts

    particleDF.to_csv(countsfile_path,index=False)